In [1]:
import importlib
import os
from models import *
from dataprocessor import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [2]:
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx", "SP00.CME.xlsx")]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test = False)
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
context_set, _ = generate_tensors(data_list, timesteps, encoder_type = "one-hot", contain_next_day_rtn=True)

生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:29<00:00,  2.70s/it]


one-hot 编码中...


生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:33<00:00,  3.01s/it]


one-hot 编码中...


In [3]:
batch_size = len(files)
# data_set = data_binder(context_set, target_set, labels, batch_size=batch_size)
# data_set.save(f"saved_data")

dataset = tf.data.Dataset.load("saved_data")

In [4]:
reload_custom_libs()
from models import *
from dataprocessor import *
from loss_functions import *

hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(files) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

In [ ]:
num_epochs = 100
print_interval = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

def train():
    
    dataset = tf.data.Dataset.load("saved_data")
    for epoch in range(num_epochs):
        dataset = dataset.shuffle(buffer_size=10000)
        iter_count = 0
        for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
            x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = batch_data
            with tf.GradientTape() as tape:
                result = model(x_c, x_c_rtn, s_c, x, s)
                loss, mle, sharpe = joint_loss_function(result, rtn_std, target_std, warm_up, alpha=1)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            iter_count += 1
            if iter_count % 1 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {loss.numpy()}, mle: {mle}, sharpe: {sharpe}")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy()}")
    
result = train()